In [32]:
from search import *
from game import *
from util import *
from pacman import *
import util, layout
import sys, types, time, random, os



In [54]:
# tiny genetic programming by © moshe sipper, www.moshesipper.com
from random import random, randint, seed
from statistics import mean
from copy import deepcopy

POP_SIZE        = 60   # population size
MIN_DEPTH       = 2    # minimal initial random tree depth
MAX_DEPTH       = 5    # maximal initial random tree depth
GENERATIONS     = 250  # maximal number of generations to run evolution
TOURNAMENT_SIZE = 5    # size of tournament for tournament selection
XO_RATE         = 0.8  # crossover rate 
PROB_MUTATION   = 0.2  # per-node mutation probability 



def add(x, y): return x + y
def sub(x, y): return x - y
def mul(x, y): return x * y

"FUNCOES ( sao as operacoes executadas nos nós das arvores)-> add, sub, mul, div (x/0 = 1)*"
"TERMINAIS ( sao os tipos possiveis de nohs folha )"
'''
    TERMINAIS ( sao os tipos possiveis de nohs folha )
        -> GAMESTATE_X, onde 0 <= X < Num. de variaveis de estado do jogo
        -> CONSTANTS ( Uns numeros constantes para algumas operações, para facilitar o calculo numerico (?) )
        
    
'''




FUNCTIONS = [add, sub, mul]
TERMINALS = ['actionScore', 'minGhostDist', 'distToNextPill', -1.0, 0.5, 1.0, 2.0] # 0.0,  -0.5,

STATE_VARS = ['actionScore', 'minGhostDist', 'distToNextPill']
" actionScore: scoreFuturo - scoreAtual "
" minGhostDist: sqrt((pacmanY - pacmanX)**2 + (ghostY - ghost)X**2) "
" distToNextPill: sqrt((pacmanY - pacmanX)**2 + (ghostY - ghost)X**2) ?????????? "


class GPTree:
    def __init__(self, data = None, left = None, right = None):
        self.data  = data
        self.left  = left
        self.right = right
        
    def setLeft(self, tree):
        self.left = tree
        
    def setRight(self, tree):
        self.right = tree
        
    def node_label(self): # string label
        if (self.data in FUNCTIONS):
            return self.data.__name__
        else: 
            return str(self.data)
    
    def print_tree(self, prefix = ""): # textual printout
        print("%s%s" % (prefix, self.node_label()))        
        if self.left:  self.left.print_tree (prefix + "   ")
        if self.right: self.right.print_tree(prefix + "   ")

    def computeTree(self, stateVars): 
        if (self.data in FUNCTIONS): 
            return self.data(self.left.compute_tree(x), self.right.compute_tree(x))
        elif (self.data in STATE_VARS): 
            return stateVars[self.data]
        else: return self.data
        
    def size(self): # tree size in nodes
        if self.data in TERMINALS: return 1
        l = self.left.size()  if self.left  else 0
        r = self.right.size() if self.right else 0
        return 1 + l + r
    
    def random_tree(self, grow, max_depth, depth = 0): # create random tree using either grow or full method
        if depth < MIN_DEPTH or (depth < max_depth and not grow): 
            self.data = FUNCTIONS[randint(0, len(FUNCTIONS)-1)]
        elif depth >= max_depth:   
            self.data = TERMINALS[randint(0, len(TERMINALS)-1)]
        else: # intermediate depth, grow
            if random () > 0.5: 
                self.data = TERMINALS[randint(0, len(TERMINALS)-1)]
            else:
                self.data = FUNCTIONS[randint(0, len(FUNCTIONS)-1)]
        if self.data in FUNCTIONS:
            self.left = GPTree()          
            self.left.random_tree(grow, max_depth, depth = depth + 1)            
            self.right = GPTree()
            self.right.random_tree(grow, max_depth, depth = depth + 1)
            
    def getSubtree(self):
        t = GPTree()
        t.data = self.data
        if self.left:  t.left  = self.left.getSubtree()
        if self.right: t.right = self.right.getSubtree()
        return t
    
    def scan_tree(self, count, second): # note: count is list, so it's passed "by reference"
        count[0] -= 1            
        if count[0] <= 1: 
            if not second: # return subtree rooted here
                return self.getSubtree()
            else: # glue subtree here
                self.data  = second.data
                self.left  = second.left
                self.right = second.right
        else:  
            ret = None              
            if self.left  and count[0] > 1: ret = self.left.scan_tree(count, second)  
            if self.right and count[0] > 1: ret = self.right.scan_tree(count, second)  
            return ret
        

In [44]:
tree = GPTree()
tree.random_tree(True, 2)
tree.print_tree()

add
   add
      minGhostDist
      -1.0
   sub
      -1.0
      minGhostDist


In [ ]:
" Criar populacao inicial"
''' Para a criação da população inicial, utilizamos o método RandomTree() que gera árvores randomicamente dentro 
    do domínio de operações fundamentais (add, sub, mul) 
        e dos terminais definidos (variaveis de estado e constantes)
'''

In [43]:
POP_SIZE = 32
N_TRAIN = 4


def initialPopulation():
    population = []
    for _ in range(32):
        tree = GPTree()
        tree.random_tree(True, 2)
        population.append(tree)
    return population

[<__main__.GPTree instance at 0x7f87b806ec80>, <__main__.GPTree instance at 0x7f87b806e960>, <__main__.GPTree instance at 0x7f87b806e780>]


In [55]:
" FITNESS "

# stateVars -> dictionary  ex stateVars['distX'] = distXGameStateVariable
def getFitness(individuo, stateVars):
    return individuo.computeTree(stateVars)

def getPopFitness(population, stateVars):
    return np.array([getFitness(population[i], stateVars) for i in range(0,len(population))])

    
    

" SELEÇÂO "
'''Para algotimo de seleção, adotaremos inicialmente o método de seleção por Torneio, na ideia de
    aumentar as randomicidade do modelo.
'''
def tournament(population, stateVars):
    indexes = np.random.choice(range(0,len(population)),4,False)
    return population[indexes[np.argmax(getPopFitness(population[indexes], stateVars))]]

def getParent(population, stateVars):
    return tournament(population, stateVars)


" CROSSOVER "
''' Para a reprodução dos pais, utilizamos uma função de crossover para árvores, 
'''

def crossover(treeA, treeB): # xo 2 trees at random nodes
#     if random() < XO_RATE:
#         geneA = treeA.scan_tree([randint(1, treeA.size())], None) # 1st random subtree
#         geneB = treeB.scan_tree([randint(1, treeB.size())], None) # 2nd random subtree
#         treeA.scan_tree([randint(1, treeA.size())], geneB) # 2nd subtree "glued" inside 1st tree
#         treeB.scan_tree([randint(1, treeB.size())], geneA) # 1st subtree "glued" inside 2nd tree
        
    child1 = treeA.getSubtree()
    child2 = treeB.getSubtree()
    
    aux = child1.right.getSubtree()
    aux.print_tree()
    child1.right = child2.left.getSubtree()
    child2.left = aux
    
    return child1, child2
    
            
" Loop de Reproducao"









In [56]:
tree = GPTree()
tree.random_tree(True, 2)
tree.print_tree()

tree2 = GPTree()
tree2.random_tree(True, 2)
tree2.print_tree()

c1, c2 = crossover(tree, tree2)
c1.print_tree()
c2.print_tree()

add
   sub
      distToNextPill
      0.5
   add
      2.0
      2.0
add
   add
      minGhostDist
      distToNextPill
   mul
      0.5
      minGhostDist
add
   2.0
   2.0
add
   sub
      distToNextPill
      0.5
   add
      minGhostDist
      distToNextPill
add
   add
      2.0
      2.0
   mul
      0.5
      minGhostDist


In [47]:
n = Directions.NORTH
s = Directions.SOUTH
e = Directions.EAST
w = Directions.WEST
actions = [w,w,w,w,s,w]

class MyGoWestAgent(Agent):
    "An agent that goes West until it can't."
    def __init__(self, tree = None):
        self.tree

    def getAction(self, state):
        "The agent receives a GameState (defined in pacman.py)."
        print(state.getLegalPacmanActions())
        print(state.getPacmanState())

        print(state.getPacmanPosition())
        print(state.getNumFood())
        print(state.getScore())
        action = s
        if len(actions) > 0:
            action = actions.pop(0)
        if action in state.getLegalPacmanActions():
            print(state.generatePacmanSuccessor(action).getGhostPositions())
            return action
        else:
            return Directions.STOP

In [48]:
args = readCommand(['--layout', 'smallClassic', '--pacman', 'GoWestAgent', '-t']) # Get game components based on input
gameLayout = layout.getLayout( 'smallClassic' )

pacman = MyGoWestAgent(ind)

ghostType = loadAgent('RandomGhost', True)
ghosts = [ghostType( i+1 ) for i in range( 4 )]

import textDisplay
textDisplay.SLEEP_TIME = 0.1
gameDisplay = textDisplay.PacmanGraphics()
# runGames( **args )
runGames( gameLayout, pacman, ghosts, gameDisplay, 1, False, numTraining = 0, catchExceptions=False, timeout=30 )

%%%%%%%%%%%%%%%%%%%%
%......%G  G%......%
%.%%...%%  %%...%%.%
%.%o.%........%.o%.%
%.%%.%.%%%%%%.%.%%.%
%........<.........%
%%%%%%%%%%%%%%%%%%%%
Score: 0

['West', 'Stop', 'East']
Pacman: (x,y)=(9, 1), Stop
(9, 1)
55
0.0
[(8, 5), (11, 5)]
%%%%%%%%%%%%%%%%%%%%
%......% GG %......%
%.%%...%%  %%...%%.%
%.%o.%........%.o%.%
%.%%.%.%%%%%%.%.%%.%
%.......> .........%
%%%%%%%%%%%%%%%%%%%%
Score: 9

['West', 'Stop', 'East']
Pacman: (x,y)=(8, 1), West
(8, 1)
54
9.0
[(9.0, 5.0), (10.0, 5.0)]
%%%%%%%%%%%%%%%%%%%%
%......% GG %......%
%.%%...%%  %%...%%.%
%.%o.%........%.o%.%
%.%%.%.%%%%%%.%.%%.%
%......>  .........%
%%%%%%%%%%%%%%%%%%%%
Score: 18

['West', 'Stop', 'East']
Pacman: (x,y)=(7, 1), West
(7, 1)
53
18.0
[(10.0, 5.0), (9.0, 5.0)]
%%%%%%%%%%%%%%%%%%%%
%......%G   %......%
%.%%...%% G%%...%%.%
%.%o.%........%.o%.%
%.%%.%.%%%%%%.%.%%.%
%.....>   .........%
%%%%%%%%%%%%%%%%%%%%
Score: 27

['West', 'Stop', 'East', 'North']
Pacman: (x,y)=(6, 1), West
(6, 1)
52
27.0
[(10.0, 4.0), (8.0, 5.0)]

[<game.Game instance at 0x7f8799f25aa0>]

In [ ]:
def trainIndividual(ind, layoutType='smallClassic'):
    gameLayout = layout.getLayout( layoutType )

    pacman = MyGoWestAgent(ind)

    ghostType = loadAgent('RandomGhost', True)
    ghosts = [ghostType( i+1 ) for i in range( 4 )]

    import textDisplay
    textDisplay.SLEEP_TIME = 0.1
    gameDisplay = textDisplay.PacmanGraphics()
    # runGames( **args )
    runGames( gameLayout, pacman, ghosts, gameDisplay, 1, False, numTraining = 0, catchExceptions=False, timeout=30 )

In [ ]:
def generationLoop():
    population = initialPopulation()
    fitness = np.zeros((1,len(population)))
    count = 1
    print("G",count)
    while count <= 200:
        # train population to get fitness
        
        for i in range(0,len(population)):
            
            
        
        children = []
        
        for i in range(0,int(population.shape[0]/2)):
            father = getParent(population)
            mother = getParent(population)
            while(np.array_equal(father, mother)):
                father = getParent(population)
            child1, child2 = crossover(father, mother)
            children.append(child1)
            children.append(child2)
            

#         children = mutation(children,int(len(children)*0.75))#mutation

        #elitism
        fittest_parent = population[np.argmax(getFitness(population))]
        children[np.argmin(getFitness(children))] = fittest_parent

        population = children.copy()
        print(children)

        fitness = getFitness(population)
        print(count, fitness)
        
        count += 1
        
    return population[np.argmax(getFitness(population))]